# library loading

In [13]:
#!pip install ko-sentence-transformers 
from sentence_transformers import SentenceTransformer, models
from ko_sentence_transformers.models import KoBertTransformer

# Model loading
word_embedding_model = KoBertTransformer("monologg/kobert", max_seq_length=75)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# embedder loading
embedder = SentenceTransformer("jhgan/ko-sbert-sts")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [5]:
# 카테고리별 corpus & embedding 불러오기
import torch
ck = torch.load('check_point.tar')

## 전체
tot_corpus = ck['tot_corpus']
tot_embedding = ck['tot_embedding']

## 기초
bas_corpus = ck['bas_corpus']
bas_embedding = ck['bas_embedding']

## 색조
col_corpus = ck['col_corpus']
col_embedding = ck['col_embedding']

In [8]:
df[0][0]

['🧡기존에 사용하던 토너가 좀 라이트 하게 느껴져서',
 '겨울용 토너로 넘버즈인 결광가득 에센스 토너를 구매하였어요',
 '✅발림성',
 '점성있는 에센스 토너임에도 마무리감이 산뜻합니다',
 '끈적임이 거의 없고 흡수가 빠릅니다',
 '✅효과',
 '▪️푸석푸석하고 거칠었던 피부가 아기 궁둥이처럼 보들보들해졌어요!',
 '▪️피부결이 좋아지면서 피부에서 광이 나요',
 '▪️피부가 팽팽해진 느낌을 받았습니다',
 '▪️세번 레이어링 하니 속건조가 잘 잡힙니다',
 '▪️전 날 에센스토너 바르고 잤더니 다음날 화장이 잘 먹더라구요',
 '▪️지인은 이 제품 3주째 바르니 피부 재생이 되어 화이트닝 효과가 있었고 잡티가 많이 개선되었다고 합니다.',
 '✅사용팁',
 '화장솜에 묻혀서 토너팩으로 활용해보세요!',
 '✅주의 사항',
 '▪️50가지 발효 성분을 담아 농축 에센스를 바른 듯',
 '얼굴에 결광을 채워주는 발효 에센스 토너로',
 '처음엔 제품이 따갑게 느껴질 수 있어요',
 '바르다보면 괜찮으니 써보세요',
 '▪️발효성분이 안맞는 분들은 테스트 후 사용하세요',
 '마음에 드는 토너였습니다',
 '추천드려요\U0001faf6',
 '💟저의 리뷰가 도움되셨다면 따봉👍 꾸욱 부탁드려요',
 '저의 다른 리뷰들도 구경하러 오세요',
 '득템을 도와드릴게요💗★ 넘버즈인 3번 결광가득 토너 ',
 '※글쓴이 피부타입 수부지 예민한 트러블 피부',
 '이제품 유튜버님이 추천영상으로 소개 시켜주셔서 캡쳐를 해두고 올영세일때 바로 구매해보았어요!! ',
 '진짜 제 피부가 엄청 예민해서 살짝만 안맞아도 바로 피부가 뒤집어져 버려서 아무제품이나 막 사용하지못하는데 이제품은 첫사용 후기 나쁘지않네요^^',
 '일단 이제품 소개를 간단히 해보자면 50가지의 발효 성분을 농축하고 있어서토너인듯 에센스인듯 피부를 단시간에 촉촉하게 물광느낌을 내주는 제품이에요 ',
 '그리고 이번에 2022 올리브영 어워즈 스킨/토너 부문에서 MD픽을 수상했다고 해요..,!',
 '

In [9]:
# data loading
df = pd.read_csv('DBdata.csv')

# function

In [14]:
# 카테고리 선택 함수
def choice_cate(no):
    num = no
    if num == 1:  # 전체
        embed = tot_embedding
        corpus = tot_corpus

    elif num == 2: # 기초
        embed = bas_embedding
        corpus = bas_corpus

    elif num == 3: # 색조
        embed = col_embedding
        corpus = col_corpus

    return embed, corpus
 
    
## 유사 문장 리스트로 반환
def list_return(query, embed, corpus):
    # 유사도 평가를 위한 작업
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embed)[0]
    cos_scores = cos_scores.cpu()
    # 상위 결과 추출
    top_results = np.argpartition(-cos_scores, range(5))[0:5]
    answer = []
    for v, idx in enumerate(top_results[0:5]):  # 산출된 결과에서
        text = corpus[idx].strip() 
        for i in range(3344): # 전체 df에서
            if text in df['rv'][i]:
                pname = df['pname'][i]
        # 리스트로 결과 저장
        answer.append([v+1, pname, text, np.round(cos_scores[idx],4)])
    return answer

## 최종 실행 함수-> 파라미터로 카테고리 숫자와 쿼리를 받아서 실행됨
def play(no, text):
    embed, corpus = choice_cate(no)
    answer = list_return(text, embed, corpus)
    return answer # answer가 리스트에 5개의 리스트요소를 담아둠

# action

In [15]:
no = int(input('카테고리를 선택하세요 :'))
tt = str(input('리뷰쿼리를 작성해주세요 :'))
answer = play(no, tt) 
print(answer) # 저장된 리스트 확인용

category2
search_review:기본기 탄탄한 크림
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 기본기 탄탄한 크림
🌼기초 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

1 [2022어워즈]에스트라 아토베리어365 크림 80ml 기획(+시카세럼10ml+패드6매) 
 ▶ ***이건 정말 찐 보습크림 !!! (Score: 0.7845)
2 [한정기획] 라로슈포제 시카플라스트 크림 B5 100ml 기획 (+시카 크림 15ml *2) 
 ▶ 보습탄력주름케어로 나온 크림인데.. (Score: 0.7803)
3 [단독기획] 넘버즈인 3번 보들보들 결 세럼 80ml 대용량 기획 
 ▶ 또 크림으로 발라주었습니다. (Score: 0.7627)
4 비욘드 엔젤아쿠아 수분 진정 크림 150ml 1+1 기획 (비건) 
 ▶ 말그대로 ‘수분’크림입니다 (Score: 0.7626)
5 [대용량+미니어처] 넘버즈인 3번 결광가득 에센스 토너 300ml 어워즈 한정기획 (+3번 결광케어 키트 5종 증정) 
 ▶ 🙋 보습력 (Score: 0.7598)

△△△△△△△△△△△△△△△△△△

[[1, '[2022어워즈]에스트라 아토베리어365 크림 80ml 기획(+시카세럼10ml+패드6매)', '***이건 정말 찐 보습크림 !!!', tensor(0.7845)], [2, '[한정기획] 라로슈포제 시카플라스트 크림 B5 100ml 기획 (+시카 크림 15ml *2)', '보습탄력주름케어로 나온 크림인데..', tensor(0.7803)], [3, '[단독기획] 넘버즈인 3번 보들보들 결 세럼 80ml 대용량 기획', '또 크림으로 발라주었습니다.', tensor(0.7627)], [4, '비욘드 엔젤아쿠아 수분 진정 크림 150ml 1+1 기획 (비건)', '말그대로 ‘수분’크림입니다', tens